# Examen ACUE Junio 2025
### Autor: Luis Ardévol Mesa

In [2]:
import pandas as pd 
import numpy as np
import csv
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

## Ejercicio 1

**Recomendación de tapas. El ejercicio consiste en predecir la valoración que el usuario “u31” realizará sobre la tapa “t1” que ofrece un determinado restaurante. Para ello dispones de dos bloques de datos: (1) las valoraciones que ha dado un conjunto de usuarios a la tapa “t1”, 2) las valoraciones del usuario “u31” sobre las tapas que ha consumido previamente. Los datos se encuentran en el fichero acue_examen_dataset.csv, y los atributos de cada registro se describen en la primera fila del fichero (“characterTapa” indica el tipo de tapa y tiene dos posibles valores: daring (atrevida) y traditional (tradicional). Se pide lo siguiente:**

- **Propón 1 método de predicción que haga uso de los datos disponibles. Explica los cálculos que hay que realizar.**

Antes de proponer un método, vamos a cargar los datos para ver cómo se disponen y qué método puede ser más conveniente en este caso

In [ ]:
file_path = './Practica/Datos/acue_examen_dataset.csv'

df = pd.read_csv(file_path, sep=',', quoting=csv.QUOTE_NONE, engine='python')
df = df.replace('"', '', regex=True)
df.columns = df.columns.str.replace('"', '')
df_tapas = df.iloc[:37].reset_index(drop=True)
df_u31 = df.iloc[38:].reset_index(drop=True)

display(df_tapas)
display(df_u31)

,tapaID,userID,restID,globalRating,tapaRating,averageRatingTapaInTapa,averageRatingRestInTapa,popularityTapa,characterTapa
0,t1,u78,r1,5.0,5.0,4.384615,4.282051,39.0,Daring
1,t1,u80,r1,5.0,5.0,4.384615,4.282051,39.0,Daring
2,t1,u81,r1,5.0,5.0,4.384615,4.282051,39.0,Daring
3,t1,u82,r1,5.0,5.0,4.384615,4.282051,39.0,Daring
4,t1,u84,r1,5.0,5.0,4.384615,4.282051,39.0,Daring
5,t1,u85,r1,5.0,5.0,4.384615,4.282051,39.0,Daring
6,t1,u89,r1,4.0,5.0,4.384615,4.282051,39.0,Daring
7,t1,u91,r1,5.0,5.0,4.384615,4.282051,39.0,Daring
8,t1,u92,r1,3.0,5.0,4.384615,4.282051,39.0,Daring
9,t1,u93,r1,5.0,5.0,4.384615,4.282051,39.0,Daring


,tapaID,userID,restID,globalRating,tapaRating,averageRatingTapaInTapa,averageRatingRestInTapa,popularityTapa,characterTapa
0,t10,u31,r6,5.0,4.0,4.085938,4.460938,128.0,Traditional
1,t102,u31,r53,5.0,5.0,3.307692,4.000000,26.0,Traditional
2,t105,u31,r54,4.0,4.0,4.361446,4.481928,83.0,Traditional
3,t107,u31,r56,5.0,4.0,3.923077,4.217949,78.0,Traditional
4,t11,u31,r7,4.0,4.0,4.496774,4.574194,155.0,Traditional
5,t13,u31,r9,5.0,5.0,3.716049,4.135802,81.0,Traditional
6,t46,u31,r25,5.0,5.0,4.522727,4.795455,44.0,Daring
7,t88,u31,r46,5.0,4.0,4.385185,4.740741,135.0,Traditional
8,t9,u31,r5,5.0,5.0,3.806818,4.000000,88.0,Traditional
9,t96,u31,r49,4.0,4.0,3.545455,3.787879,33.0,Daring


Tenemos la posibilidad de usar varios métodos, veamos cuál es el más óptimo para los datos que nos dan. El filtrado colaborativo basado en usuarios no resulta buena opción en este caso, ya que tenemos pocos datos, de modo que no hay solapamiento directo entre las valoraciones de el usuario `u31` y el resto (ni siquiera ). El filtrado colaborativo basado en items sería una buena opción si tuviésemos diversidad de tapas similares a `t1` valoradas por `u31` pero, de nuevo, no hay solapamiento. Al no haber solapamiento entre usuarios ni tapas, un enfoque colaborativo resulta ineficiente. Otra opción sería usar un modelo de regresión, pero tenemos muy pocos datos de valoraciones de u31 y demasiadas características como para ajustar un modelo medianamente robusto; en estas codiciones, la mejor opción sería un modelo que asigne la media de las valoraciones de `u31`, y una regresión Ridge o Lasso encontrarían su mejor resultado llevandose todas las características a cero/casi cero.

Con todo esto, elegimos un enfoque de **recomendación basada en contenido**, y abordaremos este método desde dos perspectivas distintas: **basado en le perfil del usuario** y **basado en similaridad entre items**. El primer enfoque nos resultará útil ya que conocemos los atributos de las tapas (`characterTapa`) y el historial de valoraciones de `u31`, con lo que podemos construir su perfil y hacer una predicción personalizada para `u31`. En el segundo enfoque intentaremos aprovechar las valoraciones de `u31` y buscar tapas similares a `t1` para dar una predicción final en base a la valoración que dio `u31` a las tapas más similares a `t1`. 

**Recomendador basado en contenido**

Como queremos hacer un recomendador basado en contenido, usaremos las características de las tapas, que serían el promedio de todas las valoraciones que recibió la tapa (`averageRatingTapaInTapa`), el promedio de todas las valoraciones que dio cada restaurante para esa tapa (`averageRatingRestInTapa`), la popularidad de la tapa (`popularityTapa`) y el tipo de tapa (`characterTapa`), que puede ser `daring` o `traditional`. Con estas características tenemos descrito cada tapa de forma unívoca. 

La tapa tendrá entonces asociado un vector de características donde tendremos los valores de las características continuas y la variable categórica `characterTapa` codificada mediante OneHotEncoding: 
$$
\text{vectorTapa} = (\text{averageRatingTapaInTapa}, \text{averageRatingRestInTapa}, \text{popularityTapa}, \text{characterTapa\_daring}, \text{characterTapa\_traditional})
$$

Debemos construir el vector de características de la tapa `t1` (sobre la que queremos hacer la predicción), y el de las tapas que valoró `u31`. 

**Perfil del usuario**

Tras esto, y para implementar el enfoque basado en el perfil del usuario, deberíamos construir el perfil de `u31` a partir de las tapas que voloró. Para esto, calculamos la media ponderada de los vectores de las tapas que `u31` valoró, y usamos la calificación que le dio a cada tapa (`tapaRating`) como peso. El perfil del usuario `u31` será un vector de características (con la misma dimensión que `vectorTapa`) que representa sus preferencias, es decir, cuáles son las características que más valora en las tapas. 
$$
\text{vectorUsuario} = \frac{\sum_{i=1}^{n} \text{tapaRating}_i \cdot \text{vectorTapa}_i}{\sum_{i=1}^{n} \text{tapaRating}_i}
$$

Con estos vectores de características calculados, solo nos queda calcular la similitud entre el vector de la tapa `t1` y el vector del usuario `u31`. Usaremos concretamente la similitud coseno:
$$
\text{similitudCoseno} = \frac{\text{vectorTapa} \cdot \text{vectorUsuario}}{\|\text{vectorTapa}\| \cdot \|\text{vectorUsuario}\|}
$$

Normalmente, los sistemas de recomendación buscan devolver una lista de items (tapas en nuestro caso) que el usuario puede disfrutar en base a su perfil, por lo que se hace un ranking en función de la similitud, pero como se nos pide dar una predicción sobre esa valoración, tenemos que buscar una forma de transformar esta similitud en una predicción de valoración. 

Al tener todos nuestros vectores componentes positivas, la similitud coseno nos dará un valor entre 0 y 1, donde valores cercanos a 1 indican que el usuario tiene preferencias muy similares a las características de la tapa `t1`, y cercanos a 0 indican que sus preferencias son opuestas. Por tnato, podemos asumir que el 0.5 es neutro, por debajo de eso habría una relación negativa, y por encima de eso una relación positiva. Con esta suposición, decimos que si la similitud es de 0.5, la predicción será la media de las valoraciones de ese usuario, si la similitud es mayor, daremos una predicción por encima de la media y, si es menor, daremos una predicción por debajo de la media. LA predicción final será
$$
\text{prediccionValoracion} = \text{averageRatingU31} + (\text{similitudCoseno} - 0.5)
$$

- **Implementa el método, realiza los cálculos, y calcula la predicción.**

Primero, hacemos un OneHotEncoding de la variable categórica `characterTapa`, para obtener `characterTapa_Daring` y `characterTapa_Traditional`.

In [27]:
encoder = OneHotEncoder()
df_u31_hot = encoder.fit_transform(df_u31[['characterTapa']]).toarray()
df_tapas_hot = encoder.transform(df_tapas[['characterTapa']]).toarray()

hot_cols = encoder.get_feature_names_out(['characterTapa'])

df_u31_hot   = pd.DataFrame(df_u31_hot,  columns=hot_cols)
df_tapas_hot = pd.DataFrame(df_tapas_hot, columns=hot_cols)

caract_cols = ['averageRatingTapaInTapa', 'averageRatingRestInTapa', 'popularityTapa']

df_u31_hot = pd.concat([df_u31[['tapaRating'] + caract_cols], df_u31_hot], axis=1)

df_tapa1_hot = pd.concat([df_tapas[['averageRatingTapaInTapa', 'averageRatingRestInTapa', 
                                    'popularityTapa']], df_tapas_hot], axis=1)

display(df_u31_hot)
display(df_tapa1_hot.head(1))

,tapaRating,averageRatingTapaInTapa,averageRatingRestInTapa,popularityTapa,characterTapa_Daring,characterTapa_Traditional
0,4.0,4.085938,4.460938,128.0,0.0,1.0
1,5.0,3.307692,4.000000,26.0,0.0,1.0
2,4.0,4.361446,4.481928,83.0,0.0,1.0
3,4.0,3.923077,4.217949,78.0,0.0,1.0
4,4.0,4.496774,4.574194,155.0,0.0,1.0
5,5.0,3.716049,4.135802,81.0,0.0,1.0
6,5.0,4.522727,4.795455,44.0,1.0,0.0
7,4.0,4.385185,4.740741,135.0,0.0,1.0
8,5.0,3.806818,4.000000,88.0,0.0,1.0
9,4.0,3.545455,3.787879,33.0,1.0,0.0


,averageRatingTapaInTapa,averageRatingRestInTapa,popularityTapa,characterTapa_Daring,characterTapa_Traditional
0,4.384615,4.282051,39.0,1.0,0.0


A continuación, para evitar que domine alguna característica concreta, como podría ser el caso de `popularityTapa`, normalizamos las variables con un min-max al intervalo [0, 1]

In [29]:
scaler = MinMaxScaler()
df_u31_hot[caract_cols] = scaler.fit_transform(df_u31_hot[caract_cols])
df_tapa1_hot[caract_cols] = scaler.transform(df_tapa1_hot[caract_cols])

display(df_u31_hot)
display(df_tapa1_hot.head(1))

,tapaRating,averageRatingTapaInTapa,averageRatingRestInTapa,popularityTapa,characterTapa_Daring,characterTapa_Traditional
0,4.0,0.640513,0.667998,0.790698,0.0,1.0
1,5.0,0.000000,0.210526,0.000000,0.0,1.0
2,4.0,0.867262,0.688831,0.441860,0.0,1.0
3,4.0,0.506475,0.426836,0.403101,0.0,1.0
4,4.0,0.978640,0.780403,1.000000,0.0,1.0
5,5.0,0.336087,0.345308,0.426357,0.0,1.0
6,5.0,1.000000,1.000000,0.139535,1.0,0.0
7,4.0,0.886800,0.945698,0.844961,0.0,1.0
8,5.0,0.410791,0.210526,0.480620,0.0,1.0
9,4.0,0.195683,0.000000,0.054264,1.0,0.0


,averageRatingTapaInTapa,averageRatingRestInTapa,popularityTapa,characterTapa_Daring,characterTapa_Traditional
0,0.886331,0.490457,0.100775,1.0,0.0


Ahora, calculamos el perfil del usuario `u31` y el vector de la tapa `t1`

In [30]:
caract_cols = caract_cols + ['characterTapa_Daring', 'characterTapa_Traditional']

ratings = df_u31_hot['tapaRating'].to_numpy()
vectores = df_u31_hot[caract_cols].to_numpy()

perfil_u31 = np.sum(vectores.T * ratings, axis=1) / np.sum(ratings)
vec_tapa1 = df_tapa1_hot.iloc[0].to_numpy()

display(perfil_u31)
display(vec_tapa1)

array([0.56899729, 0.51979233, 0.44027484, 0.20454545, 0.79545455])

array([0.88633094, 0.49045691, 0.10077519, 1.        , 0.        ])

**Predicción basada en el perfil del usuario**

Calculamos la similitud coseno y hacemos las predicciones del modo que explicamos en el apartado anterior.

In [ ]:
sim_coseno = (perfil_u31 @ vec_tapa1) / (np.linalg.norm(perfil_u31) * np.linalg.norm(vec_tapa1))
prediccion1 = df_u31['tapaRating'].mean() + (sim_coseno - 0.5)
print(f'La valoración estimada del usuario 31 a la tapa 1 será de {prediccion1:.2f}, es decir, {round(prediccion1)} estrellas')

La valoración estimada del usuario 31 a la tapa 1 será de 4.48, es decir, 4 estrellas


**Predicción basada en tapas similares**

El procedimiento es muy similar, pero en este caso no necesitamos calcular el perfil del usuario, sino que calculamos la similitud entre la tapa `t1` y las tapas que `u31` valoró, es decir, la información que tenemos en las variables `vec_tapa1` y `vectores`. 

In [32]:
similitudes = (vectores @ vec_tapa1) / (np.linalg.norm(vectores, axis=1) * np.linalg.norm(vec_tapa1))
similitudes

array([0.43372865, 0.07080612, 0.51832211, 0.38703992, 0.50127446,
       0.3006825 , 0.96420078, 0.50775164, 0.30078512, 0.80962903])

Con esas similitudes, calculamos la predicción como la media ponderada de las valoraciones de `u31` a las tapas similares, usando las similitudes como pesos. Antes comentamos que una similitud de 0 era algo completamente diferente, mientras que una similitud de 1 era algo completamente igual, por lo que podemos asumir que una similitud de 0.5 es algo neutro. Así, fijaremos el umbral de similitud en 0.5 y si la similitud es menor, asumiremos que es diferente y no la usaremos para la predicción. Si fijamos este umbal, sabemos que nos quedaremos con 5 tapas, que son concretamente t105, t11, t46, t88 y t96. 

In [ ]:
mascara = similitudes > 0.5
sim_mayores = similitudes[mascara]
ratings_mayores = ratings[mascara]

prediccion2 = np.average(ratings_mayores, weights=sim_mayores)
print(f'La valoración estimada del usuario 31 a la tapa 1 será de {prediccion2:.2f}, es decir, {round(prediccion2)} estrellas')

La valoración estimada del usuario 31 a la tapa 1 será de 4.29, es decir, 4 estrellas


Otra opción sería usar todas las tapas valoradas por `u31` para predecir. Esto lo podríamos justificar diciendo que si sabemos que un item no está para nada alineado con los gustos de usuario, nos basta con buscar características completamente opuestas para encontrar un item qeu sí encaje con las preferencias del usuario. 

In [35]:
prediccion3 = np.average(ratings, weights=similitudes)
print(f'La valoración estimada del usuario 31 a la tapa 1 será de {prediccion3:.2f}, es decir, {round(prediccion3)} estrellas')

La valoración estimada del usuario 31 a la tapa 1 será de 4.34, es decir, 4 estrellas


Con los dos enfoques de este método obtenemos resultados muy similares y, si tenemos en cuenta la naturaleza entera de la valoración (no hay un punto medio entre el 4 y el 5), tenemos un acuerdo total entre las predicciones.

- **Por último, si pudieras acceder a más datos, indica qué variables o factores adicionales, tanto de usuario como de producto, necesitarías para poder utilizarlo en la práctica. Con esos nuevos datos, ¿qué estrategia o modelo de recomendación aplicarías para resolver este problema?**

A nivel usuario, sería valioso conocer datos demográficos y económicos, permitiendo así sacar conclusiones acerca de que tipo de usuarios consume cada tipo de tapa. A nivel de producto, y siguiendo con la idea anterior, una característica muy útil sería el precio de la tapa y su tamaño, ya que muchas veces la relación cantidad-precio influye en la valoración que se hace de un producto. Además, añadir una mayor cantidad de datos nos podría servir para plantear un sistema de filtrado colaborativo basado en usuarios o en items, ya que tendríamos más datos para comparar las valoraciones de `u31` con las de otros usuarios o tapas similares.

Con las nuevas características, podríamos hacer un modelo híbrido o de regresión, que combine las características de las tapas y los usuarios. También se podría recurrir a técnicas más avanzadas de aprendizaje estadístico, pero con la cantidad de datos actual no existe la necesidad, y un modelo de esa complejidad podría tender al sobreajuste, dando peores resultados que con estos modelos simples. 

## Ejercicio 2

**Ejercicio del accidente del taxi. Suponed que ahora tenemos una tercera compañía de taxis que son de color rojo, y que la proporción de taxis es: Verde (70%), Azul (20%) y Rojo (10%) . El testigo dice que vio el taxi de color Azul e identifica correctamente un determinado color en un 80% de los casos. ¿Cuál es la probabilidad de que el taxi responsable del accidente sea Azul?**

Queremos calcular la probabilidad de que el taxi sea azul dado que el testigo dijo que era azul, es decir, necesitaremos usar el teorema de Bayes para calcular $P(\text{taxi azul} | \text{testigo dice azul})$. En principio tenemos 3 probabilidades distintas:
$$
\quad P(V) = 0.7, \quad P(A) = 0.2, \quad P(R) = 0.1
$$

Además, sabemos que el testigo acierta el color del taxi en un 80% de los casos, y se equivoca en un 20%. Queremos la probabilidad de que el taxi sea azul dado que el testigo lo ha identificado como azul, por lo que vamos a definir dos probabilidades: 
$$
P(\text{taxi azul}) = 0.2, \quad P(\text{taxi no azul}) = 0.8
$$
y, para el caso en el que el taxi no es azul, hay que distinguir entre los taxis verdes y rojos, por lo que tenemos las siguientes probabilidades condicionales:
$$
P(\text{taxi verde} | \text{taxi no azul}) = \frac{P(\text{taxi verde})}{P(\text{taxi no azul})} = \frac{0.7}{0.8} = 0.875, \qquad \qquad P(\text{taxi rojo} | \text{taxi no azul}) = \frac{P(\text{taxi rojo})}{P(\text{taxi no azul})} = \frac{0.1}{0.8} = 0.125
$$

En caso de que el testigo se equivoque, **se nos dijo que la probabilidad no depende de la proporción de taxis, simplemente del sujeto**, por lo que suponemos que puede fallar diciendo que el taxi es verde en un 50% de las veces, y diciendo que el taxi es rojo en el otro 50% de las veces. 
$$
P(\text{testigo dice azul} | \text{taxi verde}) = 0.2 \cdot \frac{1}{2} = 0.1, \qquad \qquad P(\text{testigo dice azul} | \text{taxi rojo}) = 0.2 \cdot \frac{1}{2} = 0.1
$$

Por tanto, la probabilidad de que el testigo diga que el taxi es azul, cuando realmente no es azul es:
$$
\begin{align*}
P(\text{testigo dice azul | taxi no azul}) &= P(\text{testigo dice azul} | \text{taxi verde}) \cdot P(\text{taxi verde} | \text{taxi no azul}) + P(\text{testigo dice azul} | \text{taxi rojo}) \cdot P(\text{taxi rojo} | \text{taxi no azul}) \\
&= 0.2 \Delta_V \cdot 0.875 + 0.2 \Delta_R \cdot 0.125 = 0.1
\end{align*}
$$
y la probabilidad de que el testigo diga que el taxi es azul, cuando realmente es azul es:
$$
P(\text{testigo dice azul} | \text{taxi azul}) = 0.8
$$

ya que acierta en un 80% de los casos. Con todo esto, la probabilidad total de que el testigo diga que el taxi es azul será:
$$
P(\text{testigo dice azul}) = P(\text{testigo dice azul} | \text{taxi azul}) \cdot P(\text{taxi azul}) + P(\text{testigo dice azul} | \text{taxi no azul}) \cdot P(\text{taxi no azul}) = 0.8 \cdot 0.2 + 0.1 \cdot 0.8 = 0.16 + 0.08 = 0.24
$$

Con todo esto podemos aplicar ya el teorema de Bayes: 
$$
P(\text{taxi azul} | \text{testigo dice azul}) = \frac{P(\text{testigo dice azul} | \text{taxi azul}) \cdot P(\text{taxi azul})}{P(\text{testigo dice azul})} = \frac{0.8 \cdot 0.2}{0.24} = \frac{0.16}{0.24} = \frac{2}{3} \approx 0.6667
$$

Por tanto, 
$$
\boxed{P(\text{taxi azul} | \text{testigo dice azul}) = 66.67\%}
$$


Si quisiéramos considerar la proporción de taxis de cara a la probabilidad condicional, justificando esta decisión en que, al haber distintas proporciones de taxis azules, verdes y rojos, el testigo se equivocará más en favor del color e taxis con mayor proporción, simplemente tendriamos que usar las siguientes probabilidades condicionales, recalcular $P(\text{testigo dice azul})$ y volver a aplicar el teorema de Bayes: 
- Si el taxi es azul, 
$$
\begin{align*}
P(\text{testigo dice azul} | \text{taxi azul}) &= 0.8 \\
P(\text{testigo dice verde} | \text{taxi azul}) &= 0.2 \cdot \frac{0.7}{0.7 + 0.1} = 0.175 \\
P(\text{testigo dice rojo} | \text{taxi azul}) &= 0.2 \cdot \frac{0.1}{0.7 + 0.1} = 0.025
\end{align*}
$$
- Si el taxi es verde,
$$
\begin{align*}
P(\text{testigo dice azul} | \text{taxi verde}) &= 0.2 \cdot \frac{0.2}{0.2 + 0.1} = 0.133 \\
P(\text{testigo dice verde} | \text{taxi verde}) &= 0.8 \\
P(\text{testigo dice rojo} | \text{taxi verde}) &= 0.2 \cdot \frac{0.1}{0.2 + 0.1} = 0.067
\end{align*}
$$
- Si el taxi es rojo,
$$
\begin{align*}
P(\text{testigo dice azul} | \text{taxi rojo}) &= 0.2 \cdot \frac{0.2}{0.7 + 0.2} = 0.044 \\
P(\text{testigo dice verde} | \text{taxi rojo}) &= 0.2 \cdot \frac{0.7}{0.7 + 0.2} = 0.156 \\
P(\text{testigo dice rojo} | \text{taxi rojo}) &= 0.8
\end{align*}
$$

## Ejercicio 3

**¿Cómo se estiman las preferencias del usuario en un sistema de recomendación basado en contenido? Calcula las preferencias de un usuario cuyas compras en un supermercado han sido recogidas en el fichero TT_dataset_compras.xls.**

En un sistema de recomendación basado en contenido, la idea es construir el perfil (o preferencias) de un usuario en base a los items que valoró previamente. Para ello, primero se construye el vector de características de cada item, y posteriormente se hace una media ponderada de estos, usando como pesos las valoraciones que el usuario del cuál queremos construir el perfil dio a esos items. En caso de no existir valoraciones, lo podemos hacer tomando la media aritmética. Con esto tenemos un vector único que resume los intereses de un usuario, y donde cada componente indica la utilidad de esa característica para ese usuario. 
$$
\text{perfil\_usuario} = \frac{\sum_{i=1}^{n} \text{valoracion\_item}_i \cdot \text{vector\_item}_i}{\sum_{i=1}^{n} \text{valoracion\_item}_i}
$$

Para recomendar un nuevo item, se calcula la similitud entre el perfil del usuario y el vector de característica del nuevo item, mediante, por ejemplo, la similitud coseno. La idea es presentar al usuario una lista ordenada de los items que más encajen con sus gustos, típicamente en forma de ranking y dejando que sea el usuario el que decida donde fijar el umbral, aunque también podemos devolver simplemente el top k elementos.

In [ ]:
dataset_ej5 = './Practica/Datos/TT_dataset_compras.xlsx'

df_compras = pd.read_excel(dataset_ej5, header=2, usecols=['transaction ID', 'milk', 'bread', 'butter', 'beer', 'diapers'])
df_compras

,transaction ID,milk,bread,butter,beer,diapers
0,1,1,1,0,0,0
1,2,0,0,1,0,0
2,3,0,0,0,1,1
3,4,1,1,1,0,0
4,5,0,1,0,0,0


En este caso tenemos transacciones con varios items, concretamente datos de compras realizadas en algún supermercado seguramente. Aquí es lógico no tener valoraciones y contar simplemente con el número de items de cada tipo. Por ello, podemos calcular las preferencias de este usuario haciendo la media artimética. De este modo, cada componente del vector del perfil del usuario reflejará la frecuencia relativa de cada una, con lo que podemos hacer una idea de las preferencias de este usuario.

In [13]:
perfil = df_compras.mean(axis=0)[1:].to_numpy()
print(f"El perfil/preferencias de este usuario viene dado por {perfil}")

El perfil/preferencias de este usuario viene dado por [0.4 0.6 0.4 0.2 0.2]


## Ejercicio 4

**En un proyecto de Data Science se realizan diferentes fases, comenzando por el entendimiento del problema (business understanding) y concluyendo con el despliegue final de la solución (deployment).**

- **Selecciona y justifica qué fase del proyecto : (1) es la más importante, y (2) es la que requiere más tiempo de trabajo.**

La fase **más importante** es la de **entendimiento del negocio**, ya que es dónde se define el problema y se establecen los objetivos del proyecto, los distintos KPIs, PIs, etc. Todas las decisiones posteriores, como el tratamiento de los datos, la selección de métricas y modelos, la interpretación de los resultados e incluso cómo se hará el despliegue de la aplicación, deben fundamentarse en lo establecido en la fase inicial de compresión del problema y de la necesidad de negocio, de forma que todo esté siempre alineado con esto. Una mala definición de criterios en esta fase puede llevar a un gasto de tiempo y recursos innecesario, y a unos resultados de poco valor para la empresa/desarrollador.  

Por otro lado, la fase que requiere **más tiempo** es la **preparación de los datos**, ya que se deben limpiar los datos, integrar las distintas fuentes de datos (muchas veces heterogéneas), diseñar los procesos de transformación y selección de características (si procede). Esta fase también es de gran importancia, ya que de la calidad de los datos tras esta fase dependerá en gran medida los resultados de los algoritmos/modelos aplicados posteriormente. De nuevo, una mala ejecución de esta fase puede traer problemas posteriores, por lo que hay que asegurar tanto la compatibilidad de los datos como su correcto tratamiento, de modo que los modelos posteriores sean capaces de usarlos sin dificultad o sin caer en sesgos, sobre/subajuste, etc. 

- **Si consideramos los perfiles más populares del científico de datos (ingeniero informático, estadístico, matemático, físico, economista, etc.), selecciona y justifica qué perfil o perfiles serían los más idóneos para trabajar en cada fase del proyecto.**

Vamos a analizar cada una de las fases mostradas en el diagrama: entendimiento del negocio, entendimiento de los datos, preparación de los datos, modelado, evaluación y despliegue. 

1. Entendimiento del negocio: aquí lo ideal serían los economistas, para identificar las necesidades de la empresa y definir objetivos de negocio. Esto deberíamos hacerlo en colaboración con profesionales que comprendan el problema; por ejemplo, en el caso de orientar la solución al sector sanitario, lo ideal sería la colaboración con profesionales de este sector para definir unos objetivos específicos, que puedan traer un beneficio claro. 

2. Entendimiento de los datos: los estadísticos, físicos y matemáticos tienen un perfil que se adapta muy bien a esta fase, ya que de forma general dominan el análisis exploratorio de los datos, identificaciones de patrones y distribuciones, etc. Todo esto ayuda a entender la naturaleza de los datos y ver posibles porblemas o características a explotar en el proceso posterior. Aún así, en esta fase también hay que incluir la obtención de estos datos, muchas veces accediendo a bases de datos de distinta naturaleza, usando web scrapping, etc. Para esta labor, los ingenieros informáticos resultan imprescindibles gracias a su formación.

3. Preparación de los datos: aquí los ingenieros informáticos son de gran ayuda, ya que tienen experiencia en la gestión de bases de datos, desarrollo de pipelines de transformaciones, etc. Si bien los estadísticos, físicos y matemáticos pueden apoyar en las decisiones acerca del tratamiento de los datos, todo el desarrollo formal del pipeline de ejecución debe ser computacionalmente eficiente y escalable, otro punto donde los informaticos tienen experiencia. 

4. Modelado: aquí de nuevo los físicos y mateméticos pueden aportar mucho, ya que tienen experiencia en la construcción de modelos complejos y simulaciones. De nuevo, los informáticos pueden ayudar con la implementación de los modelos, asegurando que son eficientes y escalables. Esta fase se podría ver como una colaboración de físicos y matemáticos para diseñar unos modelos adecuados, con informáticos para que esos modelos escalen bien, tengan la generalidad adecuada y sean eficientes. 

5. Evaluación: de cara a la evaluación de los modelos y comparación de los mismos, los estadísticos tienen el perfil más adecuado. No obstante, aquí también deben entrar de nuevo los perfiles que mencionamos en el entendimiento del negocio, ya que la evaluación también implica ver cómo se posicionan los resultados del modelo en el contexto de negocio, si cumple con los objetivos, etc. Por un lado, los estadísticos evalúan la bondad de los resultados, buscan el mejor modelo en base a, por ejemplo, test estadísticos y contextualizan los resultados dentro de los objetivos, mientras que, por otro lado, los perfiles de la primera fase se fijan en cómo de bien se alinean esos resultados de cara a los objetivos, ocupándose de la parte más estratégica y comercial. 

6.  Despliegue: para monitorizar el modelo en producción y asegurar su correcto funcionamiento, los ingenieros informáticos son los más adecuados. Ellos pueden encargarse de la integración del modelo en sistemas reales al tener formación y soltura en el manejo de APIs, diversos lenguajes de programación como HTML y CSS o JavaScript, necesarios para el diseño web, etc. Todo esto asegura que se cumplen las necesidades de escalabilidad y robustez.